In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd

In [ ]:
df = pd.read_csv('./Block2C_df.csv')
# df.head()

In [ ]:
# dropped unnecessary columns
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
def plot_TB_cases_by_sex():
    '''
        donut chart for showing Total TB cases for Male and Female population

        Args: None
        Returns:
            plotly express figure of donut chart
    '''
    fig = px.pie(
        data_frame=df.groupby(['Sex'])['TB_Cases'].sum().reset_index(),
        names = 'Sex',
        values = 'TB_Cases',
        hole = 0.5,
        color_discrete_sequence  = ["blue","skyblue"],
        title = 'Total TB Cases By Gender')
    fig.update_layout(template='plotly_white')
    return fig

In [ ]:
fig = plot_TB_cases_by_sex()
fig.show()

In [ ]:
def plot_TB_cases_by_year():
    '''
        plot for showing overall yearwise trend of TB cases

        Args:None
        Returns:
            plotly express figure of line chart
    '''
    fig = px.line(
        data_frame = df.groupby(['Year'])['TB_Cases'].sum().reset_index(),
        x = 'Year',
        y = 'TB_Cases',
        markers =True,
        line_shape='linear',
        title = 'TB Cases trend by Year'
        )
    fig.update_layout(
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 2021,
            dtick = 1
        ),
        template = 'plotly_white'
    )
    return fig

In [ ]:
fig = plot_TB_cases_by_year()
fig.show()

In [ ]:
def plot_genderwise_TB_cases_by_age_gr(year_quarter):
    '''
        creates a grouped bar chart showing genderwise tb_cases for each age_gr
        in a quarter.

        Args: year_quarter ==> str
            quarter of a year to analyse in the form of 2021Q1
        
        Returns:
            plotly express figure of grouped bar chart
    '''

    data = df.groupby(['Year_Quarter','Age_Gr','Sex'])['TB_Cases'].sum().reset_index()

    data = data[data['Year_Quarter']==year_quarter]
    fig = px.bar(
            data_frame = data,
            x = 'Age_Gr',
            y = 'TB_Cases',
            category_orders ={'Age_Gr': ['0 to 4','5 to 14','15 to 24','25 to 34','35 to 44','45 to 54','55 to 64','> 65']},
            color = 'Sex',
            barmode = 'group',
            title = f'Genderwise TB Cases By Age_Gr for quarter {year_quarter}'
        )
    fig.update_layout(template='plotly_white')
    return fig

In [ ]:
fig = plot_genderwise_TB_cases_by_age_gr('2022 Q2')
fig.show()

In [ ]:
def plot_TB_cases_trend_by_quarter_for_lga(lga):
    ''' 
    Generates a line graph showing trend of total TB Cases for a lga over 
    period of 2021-2023.

    Args: lga_name ==> str
            name of LGA for analysis
    
    Returns:
        plotly express figure object with a line chart
        
    '''

    data = df[df['LGA']==lga].groupby('Year_Quarter')['TB_Cases'].sum().reset_index()
    fig = px.line(
            data_frame = data,
            x = 'Year_Quarter',
            y = 'TB_Cases',
            title = f'TB Cases trend in {lga} \
                    <br><sup>between 2021-2023</sup>',
            markers = True,
            line_shape = "linear"
        )
    
    fig.update_layout(
        xaxis = dict( title = 'time period'),
        yaxis = dict( title = 'TB Cases'),
        template = 'plotly_white'
    )

    return fig

In [ ]:
fig = plot_TB_cases_trend_by_quarter_for_lga('Zaria')
fig.show()

In [ ]:
f = r'nga_adm_osgof_20190417\nga_admbnda_adm2_osgof_20190417.shp'
shapes = gpd.read_file(f)
kaduna_geo =  shapes[shapes['ADM1_EN'] == 'Kaduna']
# kaduna_geo.head()
quarterly_data =  df.groupby(['Year_Quarter','LGA'])['TB_Cases'].sum().reset_index()
quarterly_geo_data = quarterly_data.merge(kaduna_geo,left_on='LGA',right_on='ADM2_EN',how='inner')
# quarterly_geo_data.head()
quarterly_geo_data = gpd.GeoDataFrame(quarterly_geo_data)
# quarterly_geo_data.head()

In [ ]:
def plot_choropleth():
    '''
        creates choropleth map showing color range of TB cases in different LGAs
        with Year_quarter slider.

        Args: None
        Returns:
                plotly express figure of choropleth map.
    '''
    fig = px.choropleth(
                quarterly_geo_data,
                geojson=quarterly_geo_data.geometry,
                locations=quarterly_geo_data.index,
                color='TB_Cases',
                hover_name = 'ADM2_EN',
                # hover_data = 'TB_Cases',
                color_continuous_scale = 'plasma',
                projection="mercator",
                labels='Total TB Cases',
                animation_frame='Year_Quarter',
                custom_data = ['ADM2_EN','TB_Cases'],
                range_color=[0,1000])

    fig.update_layout(autosize=False,width=800,height=600)
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
        title=dict(text="Total TB Cases Over Time")
    )
    fig["layout"].pop("updatemenus")

    return fig

fig = plot_choropleth()
fig.show()